In [1]:
import numpy as np
# plot random vs nearest neighbor
import matplotlib.pyplot as plt

# generate random points
np.random.seed(0)

# non trivial 2d function to approximate
def f(x, y):
    # x,y are between 0 and 1, and the result is between 0 and 1
    return np.tanh(10 * x ), np.tanh(10 * y)

    




In [2]:
# trial history with nearest neighbors
from scipy.spatial import cKDTree

def sample_behavior_space():
    # 2 uniform random variables between 0 and 1
    x = np.random.rand()
    y = np.random.rand()
    return x, y

def sample_parameter_space():
    # 2 uniform random variables between 0 and 1
    x = np.random.rand()
    y = np.random.rand()
    return x, y

def get_nearest_neighbor(x, y, history):
    # get the nearest neighbor
    tree = cKDTree([h[1] for h in history])
    d, idx = tree.query([x, y])
    return history[idx][0]


# init history with one point
# append to history
history=[]





# bootstrap the history
for _ in range(20):
    x, y = sample_parameter_space()
    history.append([(x, y), f(x, y)])




random_history = history.copy()

In [3]:


noise = 0.05

import os


output_dir = 'frames'
os.makedirs(output_dir, exist_ok=True)


nb_trials = 1000

for trial in range(nb_trials):
    behavior_x, behavior_y = sample_behavior_space()

    # find nearest neighbor in parameter space
    param_x, param_y = get_nearest_neighbor(behavior_x, behavior_y, history)
    # add noise to the parameter space
    param_x += np.random.normal(0, noise)
    param_y += np.random.normal(0, noise)
    #clip the values
    param_x = np.abs(param_x)
    param_y = np.abs(param_y)
    # evaluate the function
    history.append([(param_x, param_y), f(param_x, param_y)])

    # sample random parameter space
    x, y = sample_parameter_space()
    random_history.append([(x, y), f(x, y)])



    # random vs nearest neighbor in two separate plots with 4 plots
    if trial % 2 == 0:
        fig, axs = plt.subplots(2, 2)

        # change size of the plot
        fig.set_size_inches(10, 10)
        # title for the whole figure in latex
        # fig.suptitle("Sampling (x,y) in parameter space and evaluating f(x,y)=tanh(10*x),tanh(10*y) in behavior space", fontsize=16)
        # same but multiple lines
        fig.suptitle("Sampling (x,y) in parameter space and\nevaluating f(x,y)", fontsize=16)
        # first plot in behavior space
        # "title":random search parameter space
        axs[1, 0].scatter([h[0][0] for h in random_history], [h[0][1] for h in random_history], s=10)
        #set bounds
        axs[1, 0].set_xlim(0, 1)
        axs[1, 0].set_ylim(0, 1)
        axs[1, 0].set_title("random search parameter sampling")
        # second plot in behavior space
        # "title":IMGEP search parameter space
        axs[1, 1].scatter([h[0][0] for h in history], [h[0][1] for h in history], s=10)
        #set bounds
        axs[1, 1].set_xlim(0, 1)
        axs[1, 1].set_ylim(0, 1)
        axs[1, 1].set_title("IMGEP parameter sampling")
        # third plot in parameter space
        # "title":random search behavior space
        axs[0, 0].scatter([h[1][0] for h in random_history], [h[1][1] for h in random_history], s=10)
        axs[0,0].set_xlim(0, 1)
        axs[0,0].set_ylim(0, 1)
        axs[0, 0].set_title("random search behavior space")
        # fourth plot in parameter space
        # "title":IMGEP search behavior space
        axs[0, 1].scatter([h[1][0] for h in history], [h[1][1] for h in history], s=10)
        axs[0, 1].set_xlim(0, 1)
        axs[0, 1].set_ylim(0, 1)
        axs[0, 1].set_title("IMGEP behavior space")

        # add subtext to the whole figure
        fig.text(0.5, 0.05, f"f(x,y)=(tanh(10*x),tanh(10*y))", ha='center', va='center', fontsize=12)

        frame_filename = os.path.join(output_dir, f'frame_{trial:04d}.png')
        plt.savefig(frame_filename)
        plt.close(fig)

        plt.close(fig)


In [254]:
# Create video from saved frames
import imageio
frames = []
frame_files = sorted([os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith('.png')])

for frame_file in frame_files:
    frames.append(imageio.imread(frame_file))

output_video = 'search_comparison.mp4'
imageio.mimsave(output_video, frames, fps=20)

print(f'Video saved as {output_video}')

/tmp/ipykernel_2886208/3024302432.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(frame_file))
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1000, 1000) to (1008, 1008) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Video saved as search_comparison.mp4
